# Applying CORDA to make new GEMs
## First we import necessary modules and define functions for later

In [1]:
import sys
import cobra.io
import corda
import pandas as pd
import re
import numpy as np
import gzip
import pickle

#Used to load pre-existing dictionaries which should be the same between different samples
def load_obj(name ):
    with open('../obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

#data from tcga follows format: SBML_id\tFPKM_value
#converts to: HGNC_id\tFPKM_value
#necessary because the genes encoded in recon2_2 follow HGNC format
def translate_gene_exp_data(InFileName):
    #import dictionary of associations between SBML gene IDs and HGNC gene IDs
    sbml_ids=load_obj('sbml_ids')
    
    #create FPKM dict with structure: FPKM[HGNC]=FPKM_value
    Data=[]
    FPKM={}
    exp=0
    
    InFile=open(InFileName,'r')
    
    for Line in InFile:
        Line=str(Line)
        Line=re.sub("'",'',Line)
        Line=re.sub("b",'',Line)
        Line=Line.strip("\\n")
        Data=Line.split("\t")
        Data[0]=re.sub('\.\d+','',Data[0])
        exp=float(Data[1])
        Level=0
        try:
            FPKM[sbml_ids[Data[0]]]=exp
        except:
            pass
        
    InFile.close()
    
    return(FPKM)

#input desired percentiles and returns list of associated values for a sample
def find_percentile_values(FPKM,h,m,l):
    #Filter through FPKM values in file
    #Based on the distribution of values NOT EQUAL TO ZERO
    #Defines confidence groups based on arguments passed in beginning of function
    
    FPKM_values=list(filter((0.0.__ne__),list(FPKM.values())))
    
    high_threshold=np.percentile(FPKM_values,h)
    medium_threshold=np.percentile(FPKM_values,m)
    low_threshold=np.percentile(FPKM_values,l)
    
    #print(low_threshold,medium_threshold,high_threshold)
    return([high_threshold,medium_threshold,low_threshold])

#generate dictionary with dict[gene]=conf_level
def assign_confidence_scores(InFileName,h,m,l):
    
    FPKM=translate_gene_exp_data(InFileName)
    thresholds=find_percentile_values(FPKM,h,m,l)
    high=thresholds[0]
    medium=thresholds[1]
    low=thresholds[2]
    
    Levels={}
    #Loops through HGNC keys in FPKM dict and creates new dict with
    #Levels[HGNC]=confidence level
    #High=3,Medium=2,Low=1,Unsure=0,Not_Detected=-1
    for i in FPKM:
        Level=0
        value=FPKM[i]
        if value>high:
            Level=3
        elif value>medium:
            Level=2
        elif value>low:
            Level=1
        #assume that values equal to exactly 0.0 were not accounted for
        elif value>0:
            Level=-1
        #print(i,'\t',value,'\t',Level)
        Levels[i]=Level
    #Levels
    return(Levels)

#convert dict[gene]=conf_level into dict[reaction]=conf
#utilizes corda.reaction_confidence from CORDA module
def make_confidence_dict(GEM,Levels):
    #generate reaction_conf dict which is creaction_conf[reaction_id]=reaction_confidence
    #confidence levels are same as gene ones, but boolean gene reaction rules are used to filter reactions that are
    #under the control of several genes
    reaction_conf={}
    for r in GEM.reactions:
        reaction_conf[r.id]=corda.reaction_confidence(r.gene_reaction_rule,Levels)
    return(reaction_conf)

def make_OutFileName(InFileName):
    #InFileName=re.sub('\/data\/NCBI\/gene\_exp\_data\/','',InFileName)
    InFileName=re.sub('\/samples\/','',InFileName)
    InFileName=InFileName.strip(".FPKM.txt")
    InFileName=InFileName+".xml"
    return(InFileName)



## Here we define the path to the data that we are using to create this GEM

In [2]:
#Specify path to data file
FileName="../samples/0349f526-7816-4a7d-9967-1f75dd9ff00a.FPKM.txt"
InFileName=FileName

## Using the gene expression data, we will now classify each gene into confidence groups
For this step, we consider the distribution of gene expression values in this sample to identify which genes are highly expressed. Other people tend to use fixed thresholds, but this ignores that the way FPKM is calculated is biased and an FPKM value of 100 could be interpreted differently in two samples.

---

# For the purposes of this document we have picked particular confidence group definitions, but we are still in the process of deciding what the best definitions are

We assume that genes with an FPKM value of exactly 0 were not detected and place them into an 'Unknown' category (Level=0)

For the remaining genes, we place genes that are above the 90th percentile in the 'High Confidence Category(Level=3). We place genes between the 90th and 70th percentile in the 'Medium Confidence' Category (Level=2) and genes between the 70th and 50th percentile in the 'Low Confidence Category' (Level=1), 

Genes below the 50th percentile, but with FPKM values not equal to exactly zero are places in a 'No Confidence' Category (Level=-1).

In [3]:
#Assign Confidence Scores with levels defined by given parameters
Levels=assign_confidence_scores(InFileName,90,70,50)

## CORDA requires a base model to function. In our case, we are using recon2.2.
This model is the most recent published model that should represent human metabolism. All of our samples are human cancer cells, so it should be an appropriate 'base model.'

In [4]:
#import RECON2 Model
recon2 = cobra.io.read_sbml_model("../GEMs/recon2_2.xml")

cobra/io/sbml.py:235 UserWarning: M_h_x appears as a reactant and product FAOXC220200x


## We will now use built in functionality from CORDA to convert our gene confidence levels into Reaction confidence levels
### These decisions are made using the boolean 'gene reaction rule' that is defined for each reaction

In [5]:
#Make confidence dict
reaction_conf = make_confidence_dict(recon2,Levels)

## met_prod is a list of reactions that CORDA must leave functional(can carry flux)
Reactions that follow the form 'metabolite -> ' are dummy reactions which represent movement inside/outside of system

In [6]:
#load list of required reactions
met_prod=load_obj('met_prod')

print(met_prod)

['3pg_c ->', 'accoa_m -> coa_m', 'akg_m ->', 'e4p_c ->', 'f6p_c ->', 'g3p_c ->', 'g6p_c ->', 'oaa_m ->', 'pep_c ->', 'pyr_c ->', 'r5p_c ->', 'succoa_m -> coa_m', 'ala_L_c ->', 'arg_L_c ->', 'asn_L_c ->', 'asp_L_c ->', 'gln_L_c ->', 'glu_L_c ->', 'gly_c ->', 'pro_L_c ->', 'ser_L_c ->', 'ctp_c ->', 'utp_c ->', 'pmtcoa_c -> coa_c', 'chsterol_c ->', 'tag_hs_c ->', 'dag_hs_c ->', 'mag_hs_c ->', 'crm_hs_c ->', 'pa_hs_c ->', 'pe_hs_c ->', 'ps_hs_c ->']


## To function, CORDA requires:
1. 'baseline' GEM
2. A dictionary of reactions and confidence levels
3. A list of reactions that we force CORDA to include in the model

For #3,I am currently using a list of reactions that were used in the original Shultz paper, but I am wondering if this step is necessary 
# Should we let the data pick which reactions must carry flux or can we reasonably expect certain reactions to always be functional?
## We have all of these, so we will run CORDA and make our GEM:

In [7]:
#initialize CORDA object
#model=recon2 'baseline' for your model, in this case recon2.2
#conf= dict of genes and confidence levels they are present in tissue of interest
# 3 = High Conf INCLUDE WHEN POSSIBLE through -1 = No Confidence EXCLUDE WHEN POSSIBLE
# 0 = unknown conf
model = corda.CORDA(recon2,reaction_conf,met_prod)
#use CORDA algorithm to construct tissue-specific model
#computationally intensive task, takes ~10 minutes
print("Building CORDA model...")
model.build()
print(model)

Building CORDA model...
build status: reconstruction complete
Inc. reactions: 2493/7817
 - unclear: 955/3197
 - exclude: 41/1190
 - low and medium: 442/2270
 - high: 1055/1160



## Now we convert our CORDA object into a COBRA object and save it for later.
Note: CORDA is the algorithm which is used to systematically remove unsupported reactions from a general model. COBRA is the toolbox which we used to analyze GEMs and run FBA and its derivatives.

In [8]:
#Manipulate InFileName for useful OutFileName for use at end of program
exp_file=make_OutFileName(InFileName)
#write COBRA model to computer
out_model=model.cobra_model()
OutFileName="../GEMs/"+exp_file
cobra.io.write_sbml_model(out_model,OutFileName)

# Overall, there are two sets of parameters that will affect the reconstruction that CORDA will produce
## 1. How we define confidence levels
Currently, we have a few sets of GEMs generated with different values
## 2. The list of reactions which CORDA is required to leave functional
All of the GEMs created thus far are using 'met_prod' from Schultz 2016 Paper